In [1]:
import os, sys

notebook_dir = os.getcwd()

#add path
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..')))

In [2]:
from scripts.preprocess_data import *

In [3]:
preprocess_data("../telegram_messages.json")

Preprocessed data saved to preprocessed_data.csv
